# Aggregate dollar values of crypto assets ("dry powder") held in all protocol-linked addresses

This uses Dune's Echo API. The query can take several hours to run so it can be run in smaller batches too. It currently checks for all the EVM chains supported by the API. Solana support in the echo API is currently in Beta(that will be added soon too).
The reason i am using this API is because i have found it to be far more reliable than dune's prices tables. 

In [ ]:
import pandas as pd
import requests 
import csv 
import warnings
from concurrent.futures import ThreadPoolExecutor
# importing some basic libraries 

In [ ]:
# make sure to "pip install dune-client" before running this
from dune_client.client import DuneClient 
dune = DuneClient("API_KEY")
query_result = dune.get_latest_result(4707829) # please remember to refresh this query for the latest results

In [ ]:
addy = dune.get_latest_result_dataframe(4707829)
test = addy['_col0'].tolist() # getting all the EVM addresses connected to FC accounts
unique_count = addy['fid'].nunique() # finding unique number of FIDs because a single FID can have mulitple address
print("number of unique FiDS -", unique_count)

Note that you can divide the 'test' into smaller lists to do it easily using small batches

# Main process

# Method 1 
This method was optimized a bit with the help of LLMs

In [ ]:
API_KEY = "API_KEY"
BASE_URL = "https://api.dune.com/api/beta/balance/"
HEADERS = {"X-Dune-Api-Key": API_KEY}
QUERYSTRING = {"filters": "native"}


In [ ]:
def is_dataframe_not_empty(df):
    return not df.empty

In [ ]:
def fetch_balance(address):
    """Fetch balance data from API for a single address."""
    url = BASE_URL + address
    response = requests.get(url, headers=HEADERS)
    
    if response.status_code == 200:
        data = response.json()
        res = data.get("balances", [])
        if res:
            df = pd.DataFrame(res)
            if is_dataframe_not_empty(df):
                if 'value_usd' in df.columns:
                    if 'low_liquidity' not in df.columns:
                        return df['value_usd'].sum()
    return 0

In [ ]:
sum_value = 0

with ThreadPoolExecutor(max_workers=10) as executor:  # Adjust workers as needed
        results = list(executor.map(fetch_balance, test))

sum_value = sum(results)
print("Total sum is:", sum_value)

In [ ]:
aval = sum_value/unique_count #
print("the average value is:", aval )

# MEthod 2
More accurate.
Much slower
Will need to restart the kernel and skip method 1 completely to avoid conflict

In [ ]:
warnings.filterwarnings("ignore", category=FutureWarning)
sum_value = 0

def is_dataframe_not_empty(df):
    return not df.empty

for i in range(len(test)):
    print("on",test[i])
    url1 ="https://api.dune.com/api/beta/balance/"
    url = url1 + test[i]
    headers = {"X-Dune-Api-Key": "p2TtjjyQnvsInPpLMqiIhFSlxhFGodxG"}
    querystring = {"filters":"native"}
    response = requests.request("GET", url, headers=headers)
    if response.status_code == 200:
        data = response.json()

        res = data["balances"]
        df1 = pd.DataFrame(res)
        if is_dataframe_not_empty(df1):
            if 'value_usd' in df1.columns:
                    if 'low_liquidity' not in df1.columns:
                        print(df1)

                        sum_value = sum_value + df1['value_usd'].sum()


print("sum is", sum_value)

In [ ]:
aval = sum_value/unique_count #
print("the average value is:", aval )